In [1]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName("Extraction_Data")\
        .config("spark.driver.extraClassPath", "/home/adriano/Documentos/airflow/jdbc/postgresql-42.7.4.jar")\
        .getOrCreate()

24/10/12 09:10:48 WARN Utils: Your hostname, Ubuntu-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/10/12 09:10:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/12 09:10:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from datetime import datetime

current_year = datetime.now().year

In [3]:
type(datetime.now().year)

int

In [4]:
ano_inicial = 2023
ano_final = 2024
for ano in range(int(ano_inicial), int(ano_final) + 1):
    print(ano)

2023
2024


In [5]:
ano_inicial = current_year
ano_final = current_year

dif_anos = (ano_final - ano_inicial) + 1
for i in range(dif_anos):
    print(ano_inicial + i)

2024


In [6]:
from pyspark.sql.functions import input_file_name, substring

origin = '/home/adriano/Documentos/airflow/database/dest/bronze/execucao-financeira'
process_all = True
ano_final = current_year
ano_inicial = current_year


if process_all == False:
    origin_file = origin + '/' + str(ano_inicial)+ '.json'
    df = spark.read.json(origin_file)
elif process_all == True:
    origin_file = origin + '/*.json'
    df = spark.read.json(origin_file) 
    df = df.withColumn('nomeArquivo',substring(input_file_name(),-9,4))


#print(df.show())
print(df.printSchema())

root
 |-- autorEmenda: string (nullable = true)
 |-- codigoAmparoLegal: long (nullable = true)
 |-- descricaoEmpenho: string (nullable = true)
 |-- fonteRecurso: string (nullable = true)
 |-- idProjetoInvestimento: string (nullable = true)
 |-- informacoesComplementares: string (nullable = true)
 |-- localEntrega: string (nullable = true)
 |-- naturezaDespesa: string (nullable = true)
 |-- nomeEsferaOrcamentaria: string (nullable = true)
 |-- nomeFavorecido: string (nullable = true)
 |-- nomeTipoEmpenho: string (nullable = true)
 |-- nrPtres: string (nullable = true)
 |-- numeroNotaEmpenhoGerada: string (nullable = true)
 |-- numeroProcesso: string (nullable = true)
 |-- pagina: long (nullable = true)
 |-- planoInterno: string (nullable = true)
 |-- planoOrcamentario: string (nullable = true)
 |-- resultadoPrimario: string (nullable = true)
 |-- tamanhoDaPagina: long (nullable = true)
 |-- tipoCredito: string (nullable = true)
 |-- ugEmitente: string (nullable = true)
 |-- ugResponsave

In [7]:
import jaydebeapi
import os
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path('/home/adriano/Documentos/airflow/.env')
load_dotenv(dotenv_path=dotenv_path)

user_dw = os.getenv('USER_DW')
password_dw = os.getenv('PASSWORD_DW')
host_dw = os.getenv('HOST_DW')
database_dw = os.getenv('DATABASE_DW')
port_dw = os.getenv('PORT_DW')
driver = "org.postgresql.Driver"

mode = 'append'
url = f'jdbc:postgresql://{host_dw}:{port_dw}/{database_dw}'
properties = {"user": user_dw, "password": password_dw, "driver": driver}
#conn = jaydebeapi.connect(driver, url, [user_dw, password_dw], '/home/adriano/Documentos/airflow/jdbc/postgresql-42.7.4.jar')
#df.write.jdbc(url=url, table='stg_execucao_financeira', mode=mode, properties=properties)





In [8]:
#curs = conn.cursor()
#curs.execute(f'''delete from stg_execucao_financeira where "nomeArquivo" = '{ano_final}' ''')

In [9]:
import requests
import time
import json

def extract_data_api(url_base, endpoint, uf, dest_path, page_size = 100, errors_limit = -1, errors_consecutives_limit = 5, executions_limit = 200):
    success = False
    page = 0
    errors_consecutives = 0
    errors = 0
    executions = 0
    method = "GET"
    dest_path_file = dest_path + '/' + str(uf) + '.json'

    Path(dest_path).mkdir(parents=True, exist_ok=True)

    if os.path.isfile(dest_path_file) == True:
        os.remove(dest_path_file)

    content_all = []

    while success == False and (errors_consecutives < errors_consecutives_limit or errors_consecutives_limit == -1) and (errors < errors_limit or errors_limit == -1) and (executions < executions_limit or executions_limit == -1):
        url = url_base + endpoint + '?' + 'pagina=' + str(page) + '&' + 'tamanhoDaPagina=' + str(page_size) + '&' + 'uf=' + str(uf)
        response = requests.request(method, url)
        if response.status_code == 200:
            if(len(response.json()["content"]) == 0):
                success = True 
            else:
                page += 1
                errors_consecutives = 0
                executions += 1
                content_all += response.json()["content"]
                #with open(dest_path_file, 'w') as f:
                #    json.dump(response.json()["content"],f)
        else:
            errors_consecutives += 1
            errors += 1
            executions += 1
            if response.status_code == 429:
                time.sleep(1)
        print(f'Status Code: {response.status_code}\n'
            f'Executions: {executions}\n'
            f'Pages: {page}\n' 
            f'N° Registers: {len(content_all)}\n'
            f'Errors: {errors}\n'
            f'Errors Consecutives: {errors_consecutives}\n')
        time.sleep(1)

    if success == True:
        print('Execution Finished with success!')
    else:
        print('Execution Finished with error!')
        if errors_consecutives < errors_consecutives_limit:
            raise Exception("Number of consecutives errors exceeded")
        elif errors < errors_limit:
            raise Exception("Number of total errors exceeded")
        elif executions < executions_limit:
            raise Exception("Number of executions exceeded")

    with open(dest_path_file, 'w', encoding='utf-8') as f:
        json.dump(content_all, f)
    f.close()


page_size = 100
url_base = "https://api.obrasgov.gestao.gov.br"
endpoint = "/obrasgov/api/projeto-investimento"
current_year = datetime.now().year
initial_year = int(current_year)
final_year = int(current_year)
dest_path = '/home/adriano/Documentos/airflow/database/dest/bronze/execucao-financeira'
errors_limit = - 1
errors_consecutives_limit = 5
executions_limit = -1

extract_data_api(url_base, endpoint, 'PR', dest_path, page_size, errors_limit, errors_consecutives_limit, executions_limit)

24/10/12 09:11:04 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
